# Para instalar una biblioteca faltante: pip install "nombre biblioteca"

In [ ]:
import numpy as np
import pandas as pd
import re as regex
import math as math
import requests as req #biblioteca para hacer peticiones HTTP a servicios: pip install request

In [ ]:
pathArchivoDataSet = 'properatti.csv'
ds = pd.read_csv(pathArchivoDataSet)

In [ ]:
ds.isnull().sum(axis=0)

# Obtener datos de lat y long a partir del geonames_id, esto no da la ubicación exacta de la propiedad, sino uno coordenada dentro de una zona (por ej. Carlos Paz o San Fernando)

### La siguiente es la URL del servicio, requiere un el nombre de un usuario generado en el sitio https://www.geonames.org/ y un geonameId, obtenido de la columna geonames_id (que vamos a ir obteniendo mientras vamos leyendo el data frame

In [ ]:
urlServicio = "http://api.geonames.org/getJSON"

### Parametros de la request

In [ ]:
PARAMS = { 'username': 'rdgagliano', 'style': 'full', 'geonameId': 3429088 }

### Realizar la request

In [ ]:
r = req.get(url = urlServicio, params = PARAMS)

In [ ]:
r.json()

### Con la respuesta podemos sacar la lat y long para completar aquellas filas del data frame que tengan el geonames_id pero no lat, long ni lat-long

### Eliminamos columnas innecesarias para el procesamiento

In [ ]:
ds.drop(ds.columns.difference(['geonames_id','lat', 'lon', 'lat-lon']), 1, inplace=True)

In [ ]:
ds.head(5)

### Solo nos quedamos con las filas en las que tengan el geonames_id

In [ ]:
ds_slice = ds.loc[~ds['geonames_id'].isnull()]

In [ ]:
ds_slice.reset_index()

### Quitamos geonames_id repetidos para no hacer peticiones innecesarias

In [ ]:
ds_slice = ds_slice.drop_duplicates(subset = ['geonames_id'], keep='first')

In [ ]:
def setLatLonFromGeonameId(dataFrameRow):    
    geonamesId = int(dataFrameRow.geonames_id)        
    
    #preparamos la peticion
    urlJson = "http://api.geonames.org/getJSON"
    parametros = { 'username': 'rdgagliano', 'style': 'full', 'geonameId': geonamesId }
    
    #realizamos la peticion
    respuesta = req.get(url = urlJson, params = parametros)
    
    #el objeto json es un diccionario
    json = respuesta.json()    
    
    try:
        dataFrameRow['lat'] = float(json['lat'])#se accede a las propiedades de json como diccionario
        dataFrameRow['lon'] = float(json['lng'])
        dataFrameRow['lat-lon'] = json['lat'] + ',' + json['lng']
    except:
        print(json)
        print(geonamesId)
    
    return dataFrameRow

In [ ]:
ds_slice = ds_slice.apply(setLatLonFromGeonameId, axis=1)

In [ ]:
ds_slice[ds_slice.lat.isnull()]

### Nos quedamos solamente con los que se pudieron completar

In [ ]:
ds_slice = ds_slice.loc[~ds['geonames_id'].isnull() & (~ds_slice['lat'].isnull() & ~ds_slice['lon'].isnull() & ~ds_slice['lat-lon'].isnull())]

# Para ahorrar el tiempo que tarda en hacer todas las requests, guardamos los datos obtenidos en un CSV para su posterior carga

### Generamos el CSV del dataframe resultante

In [ ]:
latLngCSVFileName = 'latLngFromGeonames.csv'

In [ ]:
ds_slice.to_csv(latLngCSVFileName,index=False,encoding='utf-8')

### Cargamos el archivo

In [ ]:
latLong = pd.read_csv(latLngCSVFileName)

In [ ]:
latLong.head()

# Agregamos al data frame los datos de latitud y longitud a partir del CSV generador desde geonames_id

In [ ]:
ds2 = pd.read_csv(pathArchivoDataSet)

In [ ]:
ds2.head()

In [ ]:
ds2.merge(latLong, how='left', left_on='geonames_id', right_on='geonames_id', suffixes=(False, '_geonames'))